In [12]:
# from selenium import webdriver
# import time

# driver = webdriver.Chrome()
# driver.get("https://www.vivareal.com.br/imovel/apartamento-1-quartos-jardim-goias-bairros-goiania-com-garagem-42m2-venda-RS440000-id-2536396602/")
# time.sleep(1)
# driver.execute_script('''document.getElementsByClassName('map__navigate js-navigate')[0].click()''')
# time.sleep(1)
# html_from_page = driver.page_source
# # driver.close()
# # print(html_from_page)

In [126]:
import requests

r = requests.get("https://www.vivareal.com.br/imovel/apartamento-1-quartos-jardim-goias-bairros-goiania-com-garagem-42m2-venda-RS440000-id-2536396602/")
# r = requests.get("https://www.vivareal.com.br/imoveis-lancamento/on-marista-2519073590/")
html = r.text


In [127]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

In [128]:
import re
try:
    m2 = soup.find("li", attrs={"class":"features__item features__item--area js-area"}).text
except:
    m2 = None
try:
    bedrooms = soup.find("li", attrs={"class": "features__item features__item--bedroom js-bedrooms"}).text
except:
    bedrooms = None
try:
    bathrooms = soup.find("li", attrs={"class": "features__item features__item--bathroom js-bathrooms"}).find(text=True, recursive=False)
except:
    bathrooms = None
try:
    suite = soup.find("small", attrs={"class": "features__extra-info"}).text
except:
    suite = None
try:
    condominio = soup.find("span", attrs={"class": "price__list-value condominium js-condominium"}).text
except:
    condominio = None
try:
    iptu = soup.find("span", attrs={"class": "price__list-value iptu js-iptu"}).text
except:
    iptu = None
try:
    price = soup.find("h3", attrs={"class": "price__price-info js-price-sale"}).text
except:
    price =None
try:
    parking = soup.find("li", attrs={"class": "features__item features__item--parking js-parking"}).text
except:
    parking = None
try:
    address = soup.find("p", attrs={"class": "title__address js-address"}).text
except:
    address = None
try:
    latitude = soup.find("meta", attrs={"property":"place:location:latitude"})['content']
except:
    latitude = None
try:
    longitude = soup.find("meta", attrs={"property": "place:location:longitude"})['content']
except:
    longitude = None
try:
    ads_amount = str(soup.find("script",text=re.compile("advertiserLinks")))
    teste = (ads_amount.split('''advertiserLinks":[''')[1])
    url = teste.split('''href":"''')[1].split('"')[0]
    url = "https://www.vivareal.com.br/"+url
    request_seller = requests.get(url)
    seller_soup = BeautifulSoup(request_seller.text, 'html.parser')
    amount_for_sale = int(seller_soup.find("strong", attrs={"class": "results-summary__count js-total-records"}).text)

except:
    ads_amount = None


if(amount_for_sale<=1):
    print("revenda")
else:
    print("corretora")

print(m2)
print(bedrooms)
print(bathrooms)
print(suite)
print(condominio)
print(iptu)
print(price)
print(parking)
print(address)
print(latitude)
print(longitude)

revenda
  42m²  
  1 quarto  
  1 banheiro    
 1 suíte 
 R$ 380 
 R$ 800 
   R$ 440.000   
  1 vaga  
Avenida Deputado Jamel Cecílio, 2690 - Jardim Goiás, Goiânia - GO
-16.704997
-49.240821


# Clean data

In [129]:
import re
m2_cleaned = int(re.sub("[^0-9]", "", m2))


bedrooms_cleaned = int(re.sub("[^0-9]", "", bedrooms))
bathrooms_cleaned = int(re.sub("[^0-9]", "", bathrooms))
suite_cleaned = int(re.sub("[^0-9]", "", suite))
condominio_cleaned = float(re.sub("[^0-9]", "", condominio))
iptu_cleaned = float(re.sub("[^0-9]", "", iptu))
price_cleaned = float(re.sub("[^0-9]", "", price))
parking_cleaned = int(re.sub("[^0-9]", "", parking))


print(m2_cleaned)
print(type(m2_cleaned))

print(bedrooms_cleaned)
print(type(bedrooms_cleaned))
                    
print(bathrooms_cleaned)
print(type(bathrooms_cleaned))

print(suite_cleaned)
print(type(suite_cleaned))
                    
print(condominio_cleaned)
print(type(condominio_cleaned))

print(iptu_cleaned)
print(type(iptu_cleaned))

print(price_cleaned)
print(type(price_cleaned))

print(parking_cleaned)
print(type(parking_cleaned))

42
<class 'int'>
1
<class 'int'>
1
<class 'int'>
1
<class 'int'>
380.0
<class 'float'>
800.0
<class 'float'>
440000.0
<class 'float'>
1
<class 'int'>


In [130]:

final_address, temp_address ,state  = address.split(" - ")
neighborhood, city  = temp_address.split(", ")


print(final_address)
print(neighborhood)
print(city)
print(state)

Avenida Deputado Jamel Cecílio, 2690
Jardim Goiás
Goiânia
GO
